In [ ]:
import os
import pandas as pd
import numpy as np
from pandas import DataFrame

In [ ]:
#%run ./CommonFunctions_OthersGeneral.ipynb  ## This is important for library import, getting the path and runlist

In [ ]:
## Grepping Run number (dir5) and convert only the related .ini file in dir4 into .csv file in dir3 ##

for r, d, f in os.walk(dir4):
    for num in runlist:
        datafile = (f"/Default_deeploGame_{num}.ini")
        df = pd.read_fwf(dir4 + datafile)
        newname = datafile.strip('.ini')
        df.to_csv(dir3 + str(newname) + ".csv")


In [ ]:
## List the name of screenshots(dir5) in related run into a .csv file in dir2 ##

for r, d, f in os.walk(dir5):
    for num in runlist:
        datafile = (f"/Default_deeploGame_{num}.csv")
        run_name = (f'Run{num}')
        csvfile = (dir2 + run_name + ".csv")
    
        fullscreenshotlist = []
        fullscreenshotlist_numOnly = []
        
        for filename in f:
            screenshotlist = filename
            screenshotlist = screenshotlist.strip(".bmp")
            fullscreenshotlist.append(screenshotlist)
           
            screenshotlist = screenshotlist.strip("ScreenShot").format(screenshotlist, '05d')
            fullscreenshotlist_numOnly.append(screenshotlist)
            
        d = {'Snapshot_Name':fullscreenshotlist,'Numbering_Name':fullscreenshotlist_numOnly} # have to make the 2nd column formatted as text when opening the file in excel to retain the 5-digits format
        df = pd.DataFrame(d, columns=['Snapshot_Name','Numbering_Name'])
        df.to_csv(csvfile, index=False, mode='w') 
        

In [ ]:
## Defining elements in the targeted table

for nume in runlist:
    ## Filename of desired output ##
    csvfile_table = (dir6 + "Table_" + f"Run{nume}" + ".csv")
    
    ## Count the length of TrialLockX / TrialLockY in Related Run ##
    filename_ini = (f"Default_deeploGame_{nume}.csv")
    df_ini = pd.read_csv(dir3 + filename_ini)
    count_ini = len(df_ini.index)/2

    ## Count the number of screenshot in related Run ##
    filename_screenshot = (f"Run{nume}.csv")
    df_screenshot = pd.read_csv(dir2 + filename_screenshot)
    count_screenshot = len(df_screenshot.index)
    
    ## To verify if the number of screenshots divided by length of TrialLockX / TrialLockY is tally with the 8 x 45° ##
    if count_screenshot/8 == count_ini and count_screenshot%count_ini == 0:
        pass
    elif count_screenshot/8 != count_ini and count_screenshot%count_ini != 0:
        print(f"The total numbers of {filename_screenshot} and/or columns in {filename_ini} is not tally with the 8 angles. Please check manually.")
    
    ## Getting the values of each column ##

    index_angle = {'1':'0°/360°','2':'45°','3':'90°','4':'135°','5':'180°','6':'225°','7':'270°','8':'315°'}
    totalindex = list(range(1, 9, 1)) #start number, #end_notIncluded_number, different between each 1st and 2nd number

    
    ## Column A ##
   
    # a = ViewNum # Number label of building position
    ## for every 8 rows add 1

    a = []
    
    for num in list(range(1, int(count_ini) + 1, 1)):
        for count in totalindex:
            value = num
            a.append(value)
        
        
    ## Column B and Column C ##
    
    # b = Index (loop 8 times, 1-8)
    # c = Angle (0/360, 45, 90, 135, 180, 225, 270, 315)
    
    b = []
    c = []
    
    for num in list(range(1, int(count_ini) + 1, 1)):
        for key,value in index_angle.items():
            data1 = key
            b.append(data1)
            data2 = value
            c.append(data2)
               
    
    ## Column D and Column G ##
    ## Number only, full path to ScreenShot
    
    d = []
    g = []
    
    for num in list(range(1, int(count_ini) + 1, 1)):
        for key in index_angle:
            counter_screenshot = int(num)-1+(int(count_ini)*int(key)-1) - 8158
            d.append(counter_screenshot) 
      
    d = ["{:05d}".format(item) for item in d]
    for item in d:
        counter_screenshot = (dir5 + f"Run{nume}" +"/ScreenShot" + item + ".bmp")
        g.append(counter_screenshot)
    

    ## Column E ##
    ## Have to repeat every value 8 times in a row before proceed to the next
    
    # e = X (TrialStartLocX=)
    
    e = []
          
    value_e = (df_ini.iloc[0:int(count_ini),1]) #row, column
    for words in value_e:
        TrialStartLocX = words[15:]
        TrialStartLocX = [TrialStartLocX]*8
        e.append(TrialStartLocX)
    
    e = sum(e,[])
    
    
    ## Column F ##
    ## Have to repeat every value 8 times in a row before proceed to the next
    
    # f = Y (TrialStartLocY=)

    f = []
    
    value_f = (df_ini.iloc[int(count_ini):,1])
    for words in value_f:
        TrialStartLocY = words[15:]
        TrialStartLocY = [TrialStartLocY]*8
        f.append(TrialStartLocY)

    f = sum(f,[])
    
    data = {'ViewNum':a,'Index':b,'Angle':c,'ScreenShotName':d,'X':e,'Y':f,'ScreenShotPath':g}
    df = pd.DataFrame(data, columns=['ViewNum','Index','Angle','ScreenShotName','X','Y','ScreenShotPath'])
    df.to_csv(csvfile_table, index=False, mode='w')
    